# Team Spray and Pray

<img src="http://media1.giphy.com/media/215lvo8ytxpFC/giphy.gif">

<img src='https://github.com/ga-students/DC-DSI4/blob/master/curriculum/01-week/1.01-welcome/data-science-workflow-final.jpg?raw=true'>

# 1) Identify the Problem


West Nile virus is most commonly spread to humans through infected mosquitos. Around 20% of people who become infected with the virus develop symptoms ranging from a persistent fever, to serious neurological illnesses that can result in death.

In 2002, the first human cases of West Nile virus were reported in Chicago. By 2004 the City of Chicago and the Chicago Department of Public Health (CDPH) had established a comprehensive surveillance and control program that is still in effect today.

Every week from late spring through the fall, mosquitos in traps across the city are tested for the virus. The results of these tests influence when and where the city will spray airborne pesticides to control adult mosquito populations.

Given weather, location, testing, and spraying data, __this competition asks you to predict when and where different species of mosquitos will test positive for West Nile virus.__ A more accurate method of predicting outbreaks of West Nile virus in mosquitos will help the City of Chicago and CPHD more efficiently and effectively allocate resources towards preventing transmission of this potentially deadly virus.


Submissions are evaluated on area under the ROC curve between the predicted probability that West Nile Virus is present and the observed outcomes.

Submission File

For each record in the test set, you should __predict a real-valued probability that WNV is present.__ The file should contain a header and have the following format:

    Id,WnvPresent

    1, 0

    2, 1

    3, 0.9

    4, 0.2

    etc.
    
### Strategy

We intend to merge the train, weather and spray data files and review the data where West Nile virus is present.

### Questions

What information do we need in order to predict whether or not a mosquito is + for West Nile virus (WNV)?  _Ex, do we need the spray data, weather data or only the training data?_

What species of mosquitos tested + for WNV?

What was the most common time period for mosquitos with WNV?  _Late spring - fall, per briefing, but let's try to get more specific._

What location has the most mosquitos with WNV?

# 2) Acquire the Data

We are going to start by using three files, train, spray and weather.   We will use the test data to verify which variables we are able to use in model building.

We have downloaded the files from the kaggle website and saved them on a shared git repository that Adi created.  Adi made Troy and I contributors and we forked the repository on our local drives.  

We are going to use python to complete this challenge, more specifically, we will use pandas to read, clean and perform eda.  We will also use numpy and pandas profiling.

In [1]:
# Let's import the libraries we will need at this time
import pandas as pd
import numpy as np
import pandas_profiling as pdp

# 3) Parse the Data

##### Data Description

In this competition, you will be analyzing weather data and GIS data and predicting whether or not West Nile virus is present, for a given time, location, and species. 

Every year from late-May to early-October, public health workers in Chicago setup mosquito traps scattered across the city. Every week from Monday through Wednesday, these traps collect mosquitos, and the mosquitos are tested for the presence of West Nile virus before the end of the week. The test results include the number of mosquitos, the mosquitos species, and whether or not West Nile virus is present in the cohort. 

##### Main dataset

These test results are organized in such a way that when the number of mosquitos exceed 50, they are split into another record (another row in the dataset), such that the number of mosquitos are capped at 50. 

The location of the traps are described by the block number and street name. For your convenience, we have mapped these attributes into Longitude and Latitude in the dataset. Please note that these are derived locations. For example, Block=79, and Street= "W FOSTER AVE" gives us an approximate address of "7900 W FOSTER AVE, Chicago, IL", which translates to (41.974089,-87.824812) on the map.

Some traps are "satellite traps". These are traps that are set up near (usually within 6 blocks) an established trap to enhance surveillance efforts. Satellite traps are postfixed with letters. For example, T220A is a satellite trap to T220. 

Please note that not all the locations are tested at all times. Also, records exist only when a particular species of mosquitos is found at a certain trap at a certain time. In the test set, we ask you for all combinations/permutations of possible predictions and are only scoring the observed ones.

##### Spray Data

The City of Chicago also does spraying to kill mosquitos. You are given the GIS data for their spray efforts in 2011 and 2013. Spraying can reduce the number of mosquitos in the area, and therefore might eliminate the appearance of West Nile virus.

##### Weather Data

It is believed that hot and dry conditions are more favorable for West Nile virus than cold and wet. We provide you with the dataset from NOAA of the weather conditions of 2007 to 2014, during the months of the tests. 

Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level

### File descriptions

##### train.csv, test.csv - the training and test set of the main dataset. The training set consists of data from 2007, 2009, 2011, and 2013, while in the test set you are requested to predict the test results for 2008, 2010, 2012, and 2014.

Id: the id of the record

Date: date that the WNV test is performed

Address: approximate address of the location of trap. This is used to send to the GeoCoder. 

Species: the species of mosquitos

Block: block number of address

Street: street name

Trap: Id of the trap

AddressNumberAndStreet: approximate address returned from GeoCoder

Latitude, Longitude: Latitude and Longitude returned from GeoCoder

AddressAccuracy: accuracy returned from GeoCoder

NumMosquitos: number of mosquitoes caught in this trap

WnvPresent: whether West Nile Virus was present in these mosquitos. 1 means WNV is present, and 0 means not present. 

##### spray.csv - GIS data of spraying efforts in 2011 and 2013

Date, Time: the date and time of the spray

Latitude, Longitude: the Latitude and Longitude of the spray

##### weather.csv - weather data from 2007 to 2014. 

Column descriptions in noaa_weather_qclcd_documentation.pdf.

##### sampleSubmission.csv - a sample submission file in the correct format

In [3]:
# Let's read in the files we are going to use for this competition

df_train = pd.read_csv('../assets/train.csv')
df_s = pd.read_csv('../assets/spray.csv')
df_w = pd.read_csv('../assets/weather.csv')
df_test = pd.read_csv('../assets/test.csv')
df_sample = pd.read_csv('../assets/sampleSubmission.csv')

In [11]:
# Notes for the Train data file
# Trap has 136 distinct values and Address has 138 distinct values
# 94.8% of mosquitos tested negative for WNV
# There are 813 duplicate rows
pdp.ProfileReport(df_train)

Number of variables,13
Number of observations,9693
Total Missing (%),0.0%
Total size in memory,984.5 KiB
Average record size in memory,104.0 B
Numeric,7
Categorical,6
Date,0
Text (Unique),0
Rejected,0
Distinct count,138


In [10]:
# Removed the duplicates
df_train[df_train.duplicated()].sort_values(by = 'Date')
df_train.drop_duplicates(inplace = True)

In [5]:
# Notes for the Weather data file
# DewPoint is highly correlated with Tmin
# Water1 has constant M value
pdp.ProfileReport(df_w)

Number of variables,22
Number of observations,2944
Total Missing (%),0.0%
Total size in memory,506.1 KiB
Average record size in memory,176.0 B
Numeric,5
Categorical,15
Date,0
Text (Unique),0
Rejected,2
Distinct count,178


In [44]:
# Notes for the Spray data file
# There are 541 duplicate rows
# The Time variable is missing 584 values/3.9%

pdp.ProfileReport(df_s)

Number of variables,5
Number of observations,14294
Total Missing (%),0.8%
Total size in memory,558.4 KiB
Average record size in memory,40.0 B
Numeric,3
Categorical,2
Date,0
Text (Unique),0
Rejected,0
Distinct count,10


In [13]:
# Dropped the duplicate rows
df_s.drop_duplicates(inplace = True)

In [9]:
df_train[df_train.WnvPresent == 1].describe(include=['object'])

,Date,Address,Species,Street,Trap,AddressNumberAndStreet
count,551,551,551,551,551,551
unique,53,99,3,92,97,99
top,2013-08-29,"ORD Terminal 5, O'Hare International Airport, ...",CULEX PIPIENS/RESTUANS,W OHARE AIRPORT,T900,"1000 W OHARE AIRPORT, Chicago, IL"
freq,38,66,262,66,66,66


In [10]:
df_train[df_train.WnvPresent == 1].Species.unique()

array(['CULEX PIPIENS/RESTUANS', 'CULEX PIPIENS', 'CULEX RESTUANS'], dtype=object)

In [8]:
df_w.sort_values('Date')
df_w.sort_values('Station')
df_w.Station.unique()       # return the unique values
df_w.columns
df_w.dtypes
def eda(dataframe):
    print "missing values\n", dataframe.isnull().sum()  # count how many null values are in each series
    print('')                                           # creates a blank row
    print "dataframe types \n", dataframe.dtypes        # data types
    print('')
    print "dataframe shape \n", dataframe.shape         # row count, column count
    print('')
    print "dataframe describe \n", dataframe.describe() # count, mean, stand dev, min, max, Q1, Median, Q3
    print(' ')
    print 'duplicates \n', dataframe.duplicated().sum() # True if a row is identical to a previous row
#     print 'drop dups \n', dataframe.drop_duplicates()    # Drop the duplicate rows
    print(' ')
    print('number of unique values for each column')
    for item in dataframe:
        print item
        print(' ')
        print dataframe[item].nunique()                 # number of unique values for each column

print '************************WEATHER DATAFRAME EDA**********************'
print eda(df_w)
print '*************************TRAIN DATAFRAME EDA***********************'
print eda(df_train)
print '*************************SPRAY DATAFRAME EDA***********************'
print eda(df_s)

************************WEATHER DATAFRAME EDA**********************
missing values
Station        0
Date           0
Tmax           0
Tmin           0
Tavg           0
Depart         0
DewPoint       0
WetBulb        0
Heat           0
Cool           0
Sunrise        0
Sunset         0
CodeSum        0
Depth          0
Water1         0
SnowFall       0
PrecipTotal    0
StnPressure    0
SeaLevel       0
ResultSpeed    0
ResultDir      0
AvgSpeed       0
dtype: int64

dataframe types 
Station          int64
Date            object
Tmax             int64
Tmin             int64
Tavg            object
Depart          object
DewPoint         int64
WetBulb         object
Heat            object
Cool            object
Sunrise         object
Sunset          object
CodeSum         object
Depth           object
Water1          object
SnowFall        object
PrecipTotal     object
StnPressure     object
SeaLevel        object
ResultSpeed    float64
ResultDir        int64
AvgSpeed        object
dtype:

# 4) Mine the Data

We are using the _train_ data file as our sample and we will train our models on this dataset.

We will merge the data files we plan to use, spray, weather and train.  Then perform analysis based on the merged file.

In [26]:
# Merged the dataframes together based on date

cdf = pd.merge(df_train, df_w, left_on = 'Date',    
               right_on = 'Date', indicator = True) # "indicator = True" creates a column displaying how each row 
                                                    # merged, on the left, right or both.
cdf._merge.unique()                                 # Verified that every item merged succussfully

pdp.ProfileReport(cdf)

Number of variables,34
Number of observations,19386
Total Missing (%),0.0%
Total size in memory,5.0 MiB
Average record size in memory,273.0 B
Numeric,12
Categorical,20
Date,0
Text (Unique),0
Rejected,2
Distinct count,138


In [35]:
cdf.drop('_merge', axis = 1, inplace = True) # axis=0 for rows, 1 for columns
cdf.dtypes

[both]
Categories (1, object): [both]

In [37]:
cdf = pd.merge(cdf, df_s, left_on = 'Date',         # merged the spray dataframe with the cdf dataframe
               right_on = 'Date', indicator = True) # "indicator = True" creates a column displaying how each row 
                                                    # merged, on the left, right or both.
cdf._merge.unique()

[both]
Categories (1, object): [both]

In [40]:
cdf.dtypes
pdp.ProfileReport(cdf)

Number of variables,37
Number of observations,2911126
Total Missing (%),0.0%
Total size in memory,824.6 MiB
Average record size in memory,297.0 B
Numeric,14
Categorical,18
Date,0
Text (Unique),0
Rejected,5
Distinct count,73


In [48]:
cdf.Heat.unique()
# cdf.dtypes
# df_w.Heat.unique()
df_w.Heat.value_counts()

0     1870
4       88
1       86
2       81
8       67
3       66
5       61
15      57
7       49
12      49
11      48
10      48
13      46
9       46
6       45
14      36
16      29
20      28
19      24
18      24
21      19
17      17
23      15
22      12
M       11
24       7
25       5
26       4
28       2
29       2
27       2
Name: Heat, dtype: int64